<a href="https://colab.research.google.com/github/gianmarco00/NLP_Projects/blob/main/Copy_of_Exercise_2_Word_Embeddings_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%matplotlib inline

### Source: [link](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
PATH = '/content/drive/My Drive/Colab Notebooks/NLP/'

Mounted at /content/drive


In [2]:
# Author: Robert Guthrie

import torch
#assert torch.cuda.is_available(), 'GPU not available'
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

In [3]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU is not available or not being used. Using CPU instead.")

GPU is available and being used.


In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]],
       grad_fn=<EmbeddingBackward0>)


# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.

In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss() # Negative Log Likelihood Loss
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print("Loss in Epoch {ep}: {l}".format(ep=epoch, l=np.round(total_loss, 2))) # The loss decreased every iteration over the training data!
    losses.append(total_loss)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
Loss in Epoch 0: 521.51
Loss in Epoch 1: 519.01
Loss in Epoch 2: 516.53
Loss in Epoch 3: 514.06
Loss in Epoch 4: 511.62
Loss in Epoch 5: 509.18
Loss in Epoch 6: 506.77
Loss in Epoch 7: 504.36
Loss in Epoch 8: 501.96
Loss in Epoch 9: 499.58


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.


## Exercise Layout
### 1. <u>Training CBOW Embeddings</u>
1.1) Implement a CBOW Model by completing ```class CBOW(nn.Module)``` and train it on ```raw_text```.    

1.2) Load Datasets ```tripadvisor_hotel_reviews_reduced.csv``` and ```scifi_reduced.txt```.     

1.3) Decide preprocessing steps by completing the function ```def custom_preprocess()```. Describe your decisions. Note that it's your choice to create different preprocessing functions for hotel reviews and scifi datasets or use the same preprocessing function.             

1.4) Train CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.   

1.5) Train CBOW5 with a context width of 5 (in both directions) for the Hotel Reviews dataset. Are predictions made by the model sensitive towards the context size?
     
1.6) Train CBOW2 with a context width of 2 (in both directions) for the Sci-Fi story dataset.  


### 2. <u>Test your Embeddings</u>
Note - Do the following for CBOW2, and optionally for CBOW5

2.1) For the hotel reviews dataset, choose 3 nouns, 3 verbs, and 3 adjectives. Make sure that some nouns/verbs/adjectives occur frequently in the corpus and that others are rare. For each of the 9 chosen words, retrieve the 5 closest words according to your trained CBOW2 model. List them in your report and comment on the performance of your model: do the neighbours the model provides make sense? Discuss.   

2.2) Do the same for Sci-Fi dataset.   

2.3) How does the quality of the hotel review-based embeddings compare with the Sci-fi-based embeddings? Elaborate.   

2.4) Choose 2 words and retrieve their 5 closest neighbours according to hotel review-based embeddings and the Sci-fi-based embeddings. Do they have different neighbours? If yes, can you reason why?    

2.5) What are the differences between CBOW2 and CBOW5 ? Can you "describe" them?   


### Tips

1. Switch from CPU to a GPU instance after you have confirmed that your training procedure is working correctly.
2. You can always save your intermediate results (embeddings, preprocessed dataset, model, etc.) in your google drive via colab



### 1.1 Create a CBOW Model by completing ```class CBOW(nn.Module)``` and test it on ```raw_text```
Implement CBOW in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.

In [7]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        #self.linear2 = nn.Linear(128, vocab_size)
        self.activation_fn = nn.LogSoftmax(dim=-1)


    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        #embeds = embeds.view(1, -1)
        embeds = torch.mean(embeds, dim=0).view(1, -1)

        z = self.linear1(embeds)
        out = self.activation_fn(z)
        return out

    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long)
        return self.embeddings(word).view(1, -1)
        """embeds = torch.sum(embeds, dim=0)
        out = self.linear1(embeds)
        out = self.activation_fn(out)
        return out
        pass"""

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([19, 41, 33, 24])

In [8]:
model =  CBOW(vocab_size, embedding_dim = 100)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
model.to(device)


for epoch in range(10):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)
        context_vector = context_vector.to(device)
        out = model(context_vector)

        loss = loss_function(out, torch.tensor([word_to_ix[target]], dtype=torch.long, device = device) )
        total_loss += loss.item()

        #model.zero_grad()
        #optimizer.zero_grad()
        #loss = loss_function(out, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

    print("Loss in Epoch {ep}: {l}".format(ep=epoch, l=np.round(total_loss, 2))) # The loss decreased every iteration over the training data!
    #

Loss in Epoch 0: 229.21
Loss in Epoch 1: 151.66
Loss in Epoch 2: 59.47
Loss in Epoch 3: 12.67
Loss in Epoch 4: 3.22
Loss in Epoch 5: 1.24
Loss in Epoch 6: 0.44
Loss in Epoch 7: 0.21
Loss in Epoch 8: 0.14
Loss in Epoch 9: 0.1


### 1.2 Load Datasets

In [4]:
### Load Datasets tripadvisor_hotel_reviews_reduced.csv and scifi_reduced.txt

!gdown 1foE1JuZJeu5E_4qVge9kExzhvF32teuF # For Hotel Reviews
!gdown 13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75 # For Scifi-Text

Downloading...
From: https://drive.google.com/uc?id=1foE1JuZJeu5E_4qVge9kExzhvF32teuF
To: /content/tripadvisor_hotel_reviews_reduced.csv
100% 7.36M/7.36M [00:00<00:00, 49.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75
To: /content/scifi_reduced.txt
100% 43.1M/43.1M [00:00<00:00, 127MB/s]


In [10]:
#open("tripadvisor_hotel_reviews_reduced.csv", "r").read()

### 1.3 Preprocess Datasets
### 🗒❓ Describe your decisions for preprocessing the datasets

In [5]:
### Complete the preprocessing function and apply it to the datasets

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def custom_preprocess(text, is_sci_fi=False):
    text = text.lower()
    text = "".join([char for char in text if char.isalnum() or char.isspace()]) #remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    if is_sci_fi:
        # Add custom sci-fi stop words
        stop_words.update(["spaceship", "planet", "alien"])
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens



hotel_reviews = open("tripadvisor_hotel_reviews_reduced.csv", "r").read()
sci_fi_story = open("scifi_reduced.txt", "r").read()

hotel_reviews_processed = custom_preprocess(hotel_reviews)
sci_fi_story_processed = custom_preprocess(sci_fi_story, is_sci_fi=True)

print(len(hotel_reviews_processed))
print(len(sci_fi_story_processed))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


1023892
3954325


In [6]:
vocab1 = set(hotel_reviews_processed)

vocab_size1 = len(vocab1)
print(vocab_size1)

word_to_ix = {word: i for i, word in enumerate(vocab1)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

data2 = []
for i in range(2, len(hotel_reviews_processed) - 2):
    context2 = [hotel_reviews_processed[i - 2], hotel_reviews_processed[i - 1],
               hotel_reviews_processed[i + 1], hotel_reviews_processed[i + 2]]
    target2 = hotel_reviews_processed[i]
    data2.append((context2, target2))
print(data2[:5])

data5 = []
for j in range(5, len(hotel_reviews_processed) - 5):
    context5 = [hotel_reviews_processed[j - 5], hotel_reviews_processed[j - 4], hotel_reviews_processed[j - 3], hotel_reviews_processed[j - 2], hotel_reviews_processed[j - 1],
               hotel_reviews_processed[j + 1], hotel_reviews_processed[j + 2], hotel_reviews_processed[j + 3], hotel_reviews_processed[j + 4], hotel_reviews_processed[j + 5]]
    target5 = hotel_reviews_processed[j]
    data5.append((context5, target5))
print(data5[:5])



datatest2 = data2[0:10000]
datatest5 = data5[0:10000]

print(len(data5))


49050
[(['reviewrating', 'fantastic', 'large', 'hotel'], 'service'), (['fantastic', 'service', 'hotel', 'caters'], 'large'), (['service', 'large', 'caters', 'business'], 'hotel'), (['large', 'hotel', 'business', 'corporates'], 'caters'), (['hotel', 'caters', 'corporates', 'serve'], 'business')]
[(['reviewrating', 'fantastic', 'service', 'large', 'hotel', 'business', 'corporates', 'serve', 'provided', 'better'], 'caters'), (['fantastic', 'service', 'large', 'hotel', 'caters', 'corporates', 'serve', 'provided', 'better', 'wife'], 'business'), (['service', 'large', 'hotel', 'caters', 'business', 'serve', 'provided', 'better', 'wife', 'experienced'], 'corporates'), (['large', 'hotel', 'caters', 'business', 'corporates', 'provided', 'better', 'wife', 'experienced', 'nothing'], 'serve'), (['hotel', 'caters', 'business', 'corporates', 'serve', 'better', 'wife', 'experienced', 'nothing', 'short'], 'provided')]
1023882


### 1.4 Train CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class CBOW2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW2, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128) # Modified: Input size should match embedding_dim
        self.dropout = nn.Dropout(0.25)  # Dropout layer for regularization
        self.activation_fn = nn.ReLU()  # Activation function (ReLU in this case)
        self.linear2 = nn.Linear(128, vocab_size)
        self.context_size = context_size
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        embeds = torch.mean(embeds, dim=1).view(inputs.shape[0], -1) # take mean on the context, keep batch dim
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        embeds = self.dropout(embeds)
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        out = self.linear1(embeds)
        #print(f"out shape: {out.shape}") #DEBUG
        out = self.activation_fn(out)
        #print(f"out shape: {out.shape}") #DEBUG
        out = self.linear2(out)
        #print(f"out shape: {out.shape}") #DEBUG
        log_probs = F.log_softmax(out, dim=-1)
        return log_probs


    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long).to(device)
        return self.embeddings(word).view(1, -1)

def make_context_vector(context, word_to_ix, context_size):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long).to(device)


In [19]:
#model.to(device)

In [8]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set CUDA launch blocking for debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Check CUDA availability
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Current Device: {torch.cuda.current_device()}")
print(f"Device Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

# Clear CUDA cache
torch.cuda.empty_cache()

# 1. Model Initialization
model1 = CBOW2(vocab_size1, embedding_dim=50, context_size=2)  # Added context_size
model1.to(device)  # Move model to the appropriate device (CPU or GPU)

# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
#optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)

# 3. Prepare Data for DataLoader (Modified)
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(30):
    total_loss = 0
    for i in range(0, len(data2), 128): # Iterate in batches of 128
        context_batch = []
        target_batch = []
        for context2, target2 in data2[i:i+128]: # Get data for current batch
            context_vector = make_context_vector(context2, word_to_ix, 2)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix[target2])

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model1(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    avg_loss = total_loss / (len(data2) // 128 + (len(data2) % 128 > 0)) # Adjust for incomplete batches

    scheduler.step(avg_loss)  #printing the learning rates to know which one is optimal - scheduler stuff for each epoch
    current_lr = scheduler.optimizer.param_groups[0]['lr']

    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, avg_loss {np.round(avg_loss, 5)}, Learning rate after Epoch {epoch}: {current_lr}")








CUDA Available: True
Current Device: 0
Device Count: 1
Device Name: Tesla T4
Loss in Epoch 0: 61792.65, avg_loss 7.72408, Learning rate after Epoch 0: 0.001
Loss in Epoch 1: 58012.45, avg_loss 7.25156, Learning rate after Epoch 1: 0.001
Loss in Epoch 2: 56359.65, avg_loss 7.04496, Learning rate after Epoch 2: 0.001
Loss in Epoch 3: 55343.88, avg_loss 6.91799, Learning rate after Epoch 3: 0.001
Loss in Epoch 4: 54615.16, avg_loss 6.8269, Learning rate after Epoch 4: 0.001
Loss in Epoch 5: 54092.71, avg_loss 6.76159, Learning rate after Epoch 5: 0.001
Loss in Epoch 6: 53713.82, avg_loss 6.71423, Learning rate after Epoch 6: 0.001
Loss in Epoch 7: 53378.58, avg_loss 6.67232, Learning rate after Epoch 7: 0.001
Loss in Epoch 8: 53131.35, avg_loss 6.64142, Learning rate after Epoch 8: 0.001
Loss in Epoch 9: 52919.96, avg_loss 6.61499, Learning rate after Epoch 9: 0.001
Loss in Epoch 10: 52752.15, avg_loss 6.59402, Learning rate after Epoch 10: 0.001
Loss in Epoch 11: 52604.93, avg_loss 6.575

In [13]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR, OneCycleLR

model1 = CBOW2(vocab_size1, embedding_dim=50, context_size=2)  # Added context_size
model1.to(device)  # Move model to the appropriate device (CPU or GPU)

# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()


# Initial high learning rate with gradual warm-up and cosine annealing
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# Sequential scheduler: Linear warm-up followed by CosineAnnealing
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.005, steps_per_epoch=len(data2) // 128, epochs=30
)

# 3. Prepare Data for DataLoader (Modified)
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(30):
    total_loss = 0
    for i in range(0, len(data2), 128): # Iterate in batches of 128
        context_batch = []
        target_batch = []
        for context2, target2 in data2[i:i+128]: # Get data for current batch
            context_vector = make_context_vector(context2, word_to_ix, 2)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix[target2])

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model1(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    avg_loss = total_loss / (len(data2) // 128 + (len(data2) % 128 > 0))
    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, Average loss {np.round(avg_loss, 5)}")

    print(f"Lr before stepping: {optimizer.param_groups[0]['lr']}")
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning rate after Epoch {epoch}: {current_lr}")


Loss in Epoch 0: 62913.52, Average loss 7.86419
Lr before stepping: 0.00019999999999999966
Learning rate after Epoch 0: 0.00020000000228526538
Loss in Epoch 1: 59631.74, Average loss 7.45397
Lr before stepping: 0.00020000000228526538
Learning rate after Epoch 1: 0.00020000000914106083
Loss in Epoch 2: 58422.03, Average loss 7.30275
Lr before stepping: 0.00020000000914106083
Learning rate after Epoch 2: 0.0002000000205673886
Loss in Epoch 3: 57476.16, Average loss 7.18452
Lr before stepping: 0.0002000000205673886
Learning rate after Epoch 3: 0.00020000003656424608
Loss in Epoch 4: 56714.86, Average loss 7.08936
Lr before stepping: 0.00020000003656424608
Learning rate after Epoch 4: 0.00020000005713163415
Loss in Epoch 5: 56080.99, Average loss 7.01012
Lr before stepping: 0.00020000005713163415
Learning rate after Epoch 5: 0.00020000008226955367
Loss in Epoch 6: 55540.93, Average loss 6.94262
Lr before stepping: 0.00020000008226955367
Learning rate after Epoch 6: 0.00020000011197800292
L

### 1.5 Train CBOW5 with a context width of 5 (in both directions) for the Hotel Reviews dataset.  

🗒❓ Are predictions made by the model sensitive towards the context size?

In [14]:
model2 = CBOW2(vocab_size1, embedding_dim=50, context_size=5)  # Added context_size
model2.to(device)  # Move model to the appropriate device (CPU or GPU)5
# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)
# 3. Prepare Data for DataLoader (Modified)
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(30):
    total_loss = 0
    for i in range(0, len(data5), 128): # Iterate in batches of 128
        context_batch = []
        target_batch = []
        for context5, target5 in data5[i:i+128]: # Get data for current batch
            context_vector = make_context_vector(context5, word_to_ix, 5)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix[target5])

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model2(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    avg_loss = total_loss / (len(data5) // 128 + (len(data5) % 128 > 0)) # Adjust for incomplete batches
    scheduler.step(avg_loss)
    current_lr = scheduler.optimizer.param_groups[0]['lr']
    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, Average loss {np.round(avg_loss, 5)}, Learning rate after Epoch {epoch}: {current_lr}")





Loss in Epoch 0: 61810.11, Average loss 7.72626, Learning rate after Epoch 0: 0.001
Loss in Epoch 1: 58303.25, Average loss 7.28791, Learning rate after Epoch 1: 0.001
Loss in Epoch 2: 56961.59, Average loss 7.1202, Learning rate after Epoch 2: 0.001
Loss in Epoch 3: 56099.93, Average loss 7.01249, Learning rate after Epoch 3: 0.001
Loss in Epoch 4: 55497.26, Average loss 6.93716, Learning rate after Epoch 4: 0.001
Loss in Epoch 5: 55060.04, Average loss 6.88251, Learning rate after Epoch 5: 0.001
Loss in Epoch 6: 54701.7, Average loss 6.83771, Learning rate after Epoch 6: 0.001
Loss in Epoch 7: 54433.49, Average loss 6.80419, Learning rate after Epoch 7: 0.001
Loss in Epoch 8: 54220.86, Average loss 6.77761, Learning rate after Epoch 8: 0.001
Loss in Epoch 9: 54069.0, Average loss 6.75862, Learning rate after Epoch 9: 0.001
Loss in Epoch 10: 53935.38, Average loss 6.74192, Learning rate after Epoch 10: 0.001
Loss in Epoch 11: 53755.16, Average loss 6.7194, Learning rate after Epoch 11

In [15]:
model2 = CBOW2(vocab_size1, embedding_dim=50, context_size=5)  # Added context_size
model2.to(device)  # Move model to the appropriate device (CPU or GPU)5
# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)


scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
# 3. Prepare Data for DataLoader (Modified)
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(30):
    total_loss = 0
    for i in range(0, len(data5), 128): # Iterate in batches of 128
        context_batch = []
        target_batch = []
        for context5, target5 in data5[i:i+128]: # Get data for current batch
            context_vector = make_context_vector(context5, word_to_ix, 5)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix[target5])

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model2(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    avg_loss = total_loss / (len(data5) // 128 + (len(data5) % 128 > 0)) # Adjust for incomplete batches
    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, Average loss {np.round(avg_loss, 5)}")

    print(f"Lr before stepping: {optimizer.param_groups[0]['lr']}")
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning rate after Epoch {epoch}: {current_lr}")

Loss in Epoch 0: 61833.32, Average loss 7.72917
Lr before stepping: 0.001
Learning rate after Epoch 0: 0.001
Loss in Epoch 1: 58360.29, Average loss 7.29504
Lr before stepping: 0.001
Learning rate after Epoch 1: 0.001
Loss in Epoch 2: 57035.37, Average loss 7.12942
Lr before stepping: 0.001
Learning rate after Epoch 2: 0.001
Loss in Epoch 3: 56187.96, Average loss 7.02349
Lr before stepping: 0.001
Learning rate after Epoch 3: 0.001
Loss in Epoch 4: 55591.84, Average loss 6.94898
Lr before stepping: 0.001
Learning rate after Epoch 4: 0.0005
Loss in Epoch 5: 55144.0, Average loss 6.893
Lr before stepping: 0.0005
Learning rate after Epoch 5: 0.0005
Loss in Epoch 6: 54866.23, Average loss 6.85828
Lr before stepping: 0.0005
Learning rate after Epoch 6: 0.0005
Loss in Epoch 7: 54654.29, Average loss 6.83179
Lr before stepping: 0.0005
Learning rate after Epoch 7: 0.0005
Loss in Epoch 8: 54481.15, Average loss 6.81014
Lr before stepping: 0.0005
Learning rate after Epoch 8: 0.0005
Loss in Epoch

In [16]:
import torch.nn.functional as F

# Function to get the 5 closest words
def get_closest_words(word, word_to_ix, embeddings, top_n=5):
    word_idx = word_to_ix[word]
    word_embedding = embeddings(torch.tensor([word_idx], device=device))

    # Compute cosine similarities
    similarities = {}
    for other_word, other_idx in word_to_ix.items():
        if other_word != word:
            other_embedding = embeddings(torch.tensor([other_idx], device=device))
            cos_sim = F.cosine_similarity(word_embedding, other_embedding).item()
            similarities[other_word] = cos_sim

    # Sort by similarity and return top_n
    closest_words = sorted(similarities, key=similarities.get, reverse=True)[:top_n]
    return closest_words

# Words to test
nouns = ['hotel', 'room', 'service']
verbs = ['stay', 'enjoy', 'recommend']
adjectives = ['clean', 'comfortable', 'friendly']

# Get embeddings from the model
embeddings = model1.embeddings

# Find and print closest words
for category, words in [('Nouns', nouns), ('Verbs', verbs), ('Adjectives', adjectives)]:
    print(f"\n{category}:")
    for word in words:
        closest = get_closest_words(word, word_to_ix, embeddings)
        print(f"{word}: {closest}")


Nouns:
hotel: ['doublecheck', 'courtesyhaving', 'property', 'unparalled', 'comforted']
room: ['31c', 'unit', 'intranet', 'expo', 'everybodythe']
service: ['fruityogurt', 'kogarah', 'transplant', 'bitsi', 'equally']

Verbs:
stay: ['experience', 'republique', 'fri', 'cityand', 'secorely']
enjoy: ['detox', '2500day', 'overground', 'caribbean', 'stumble']
recommend: ['recommended', 'recomend', 'sloane', 'factor2', 'picturesin']

Adjectives:
clean: ['spotless', 'contemporary', 'disappointment', '1005', 'nonexistant']
comfortable: ['comfy', 'linen', 'kingsize', 'inwould', 'sagged']
friendly: ['friendlythe', 'cheerful', 'courteous', 'trained', 'hotellocated']


### 1.6 Train CBOW2 with a context width of 2 (in both directions) for the Sci-Fi story dataset

In [17]:
#vocab1 = set(hotel_reviews_processed)
vocab2 = set(sci_fi_story_processed)
vocab_size2 = len(vocab2)
print(vocab_size2)

word_to_ix2 = {word: i for i, word in enumerate(vocab2)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

dataS2 = []
for i in range(2, len(sci_fi_story_processed) - 2):
    contextS2 = [sci_fi_story_processed[i - 2], sci_fi_story_processed[i - 1],
               sci_fi_story_processed[i + 1], sci_fi_story_processed[i + 2]]
    targetS2 = sci_fi_story_processed[i]
    dataS2.append((contextS2, targetS2))
print(dataS2[:5])
print(len(dataS2))

datatestS2 = dataS2[0:10000]

print(len(datatestS2))


121871
[(['chat', 'editor', 'fiction', 'magazine'], 'science'), (['editor', 'science', 'magazine', 'called'], 'fiction'), (['science', 'fiction', 'called', 'title'], 'magazine'), (['fiction', 'magazine', 'title', 'selected'], 'called'), (['magazine', 'called', 'selected', 'much'], 'title')]
3954321
10000


In [18]:

class CBOWS2(nn.Module):
    def __init__(self, vocab_size2, embedding_dim, context_size):
        super(CBOWS2, self).__init__()
        self.embeddings = nn.Embedding(vocab_size2, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128) # Modified: Input size should match embedding_dim
        self.dropout = nn.Dropout(0.25)  # Dropout layer for regularization
        self.activation_fn = nn.ReLU()  # Activation function (ReLU in this case)
        self.linear2 = nn.Linear(128, vocab_size2)
        self.context_size = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        embeds = torch.mean(embeds, dim=1).view(inputs.shape[0], -1) # take mean on the context, keep batch dim
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        embeds = self.dropout(embeds)
        #print(f"embeds shape: {embeds.shape}") #DEBUG
        out = self.linear1(embeds)
        #print(f"out shape: {out.shape}") #DEBUG
        out = self.activation_fn(out)
        #print(f"out shape: {out.shape}") #DEBUG
        out = self.linear2(out)
        #print(f"out shape: {out.shape}") #DEBUG
        log_probs = F.log_softmax(out, dim=-1)
        return log_probs


    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix2[word]], dtype=torch.long).to(device)
        return self.embeddings(word).view(1, -1)

def make_context_vector(context, word_to_ix, context_size):
    idxs = [word_to_ix2[w] for w in contextS2]
    return torch.tensor(idxs, dtype=torch.long).to(device)

In [19]:



model3 = CBOWS2(vocab_size2, embedding_dim=30, context_size=2)  # Added context_size
model3.to(device)  # Move model to the appropriate device (CPU or GPU)5
# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()
optimizer = optim.AdamW(model3.parameters(), lr=0.01)

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1)
scheduler = StepLR(optimizer, step_size=2, gamma=0.95)

# 3. Prepare Data for DataLoader (Modified1
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(20):
    total_loss = 0
    #for i in range(0, len(dataS2), 128):# Iterate in batches of 128
    for i in range(0, len(dataS2), 512):
        context_batch = []
        target_batch = []
        for contextS2, targetS2 in dataS2[i:i+512]:#[i:i+128] # Get data for current batch
            context_vector = make_context_vector(contextS2, word_to_ix2, 2)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix2[targetS2]) #

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model3(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    #avg_loss = total_loss / (len(dataS2) // 128 + (len(dataS2) % 128 > 0))
    avg_loss = total_loss / (len(dataS2) // 512 + (len(dataS2) % 512 > 0))# Adjust for incomplete batches

    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, Average loss {np.round(avg_loss, 5)}")

    print(f"Lr before stepping: {optimizer.param_groups[0]['lr']}")
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning rate after Epoch {epoch}: {current_lr}")

Loss in Epoch 0: 68359.18, Average loss 8.85023
Lr before stepping: 0.01
Learning rate after Epoch 0: 0.01
Loss in Epoch 1: 65961.02, Average loss 8.53975
Lr before stepping: 0.01
Learning rate after Epoch 1: 0.0095
Loss in Epoch 2: 65923.2, Average loss 8.53485
Lr before stepping: 0.0095
Learning rate after Epoch 2: 0.0095
Loss in Epoch 3: 65595.37, Average loss 8.49241
Lr before stepping: 0.0095
Learning rate after Epoch 3: 0.009025
Loss in Epoch 4: 65674.9, Average loss 8.50271
Lr before stepping: 0.009025
Learning rate after Epoch 4: 0.009025
Loss in Epoch 5: 65410.08, Average loss 8.46842
Lr before stepping: 0.009025
Learning rate after Epoch 5: 0.00857375
Loss in Epoch 6: 65314.5, Average loss 8.45605
Lr before stepping: 0.00857375
Learning rate after Epoch 6: 0.00857375
Loss in Epoch 7: 65319.34, Average loss 8.45667
Lr before stepping: 0.00857375
Learning rate after Epoch 7: 0.0081450625
Loss in Epoch 8: 65311.41, Average loss 8.45565
Lr before stepping: 0.0081450625
Learning r

In [ ]:



model4 = CBOWS2(vocab_size2, embedding_dim=20, context_size=2)  # Added context_size
model4.to(device)  # Move model to the appropriate device (CPU or GPU)5
# 2. Loss Function and Optimizer Setup
loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model4.parameters(), lr=0.01)

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1)
scheduler = StepLR(optimizer, step_size=4, gamma=0.95)

# 3. Prepare Data for DataLoader (Modified1
# Instead of creating large tensors upfront, we will create them in batches within the loop
# This reduces the memory footprint

# 4. Training Loop (Modified)
for epoch in range(20):
    total_loss = 0
    #for i in range(0, len(dataS2), 128):# Iterate in batches of 128
    for i in range(0, len(dataS2), 1024):
        context_batch = []
        target_batch = []
        for contextS2, targetS2 in dataS2[i:i+1024]:#[i:i+128] # Get data for current batch
            context_vector = make_context_vector(contextS2, word_to_ix2, 2)
            context_batch.append(context_vector)
            target_batch.append(word_to_ix2[targetS2]) #

        # Convert to tensors for current batch
        context_batch = torch.stack(context_batch).to(device)
        target_batch = torch.tensor(target_batch, device=device) # Create tensor directly on device

        optimizer.zero_grad()
        out = model4(context_batch)
        loss = loss_func(out, target_batch)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model4.parameters(), max_norm=1.0) #gradient clipping
        optimizer.step()

    #avg_loss = total_loss / (len(dataS2) // 128 + (len(dataS2) % 128 > 0))
    avg_loss = total_loss / (len(dataS2) // 1024 + (len(dataS2) % 1024 > 0))# Adjust for incomplete batches

    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}, Average loss {np.round(avg_loss, 5)}")

    print(f"Lr before stepping: {optimizer.param_groups[0]['lr']}")
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning rate after Epoch {epoch}: {current_lr}")

Loss in Epoch 0: 34090.1, Average loss 8.82706
Lr before stepping: 0.01
Learning rate after Epoch 0: 0.01
Loss in Epoch 1: 32897.9, Average loss 8.51836
Lr before stepping: 0.01
Learning rate after Epoch 1: 0.01
Loss in Epoch 2: 32648.88, Average loss 8.45388
Lr before stepping: 0.01
Learning rate after Epoch 2: 0.01
Loss in Epoch 3: 32565.59, Average loss 8.43231
Lr before stepping: 0.01
Learning rate after Epoch 3: 0.0095
Loss in Epoch 4: 32438.16, Average loss 8.39932
Lr before stepping: 0.0095
Learning rate after Epoch 4: 0.0095


### 2.1 For the hotel reviews dataset, choose 3 nouns, 3 verbs, and 3 adjectives. (CBOW2 and optionally for CBOW5)
Make sure that some nouns/verbs/adjectives occur frequently in the corpus and that others are rare. For each of the 9 chosen words, retrieve the 5 closest words according to your trained CBOW2 model.    

🗒❓ List them in your report (at the end of this notebook) and comment on the performance of your model: do the neighbours the model provides make sense? Discuss.   


### 2.2 Repeat 2.1 for SciFi Dataset

🗒❓ List your findings for SciFi Dataset as well, similarly to 2.1

In [20]:

def get_closest_words1(word, word_to_ix2, embeddings, top_n=5):
    word_idx = word_to_ix2[word]
    word_embedding = embeddings(torch.tensor([word_idx], device=device))

    # Compute cosine similarities
    similarities = {}
    for other_word, other_idx in word_to_ix2.items():
        if other_word != word:
            other_embedding = embeddings(torch.tensor([other_idx], device=device))
            cos_sim = F.cosine_similarity(word_embedding, other_embedding).item()
            similarities[other_word] = cos_sim

    # Sort by similarity and return top_n
    closest_words = sorted(similarities, key=similarities.get, reverse=True)[:top_n]
    return closest_words

# Words to test
nouns = ['robot', 'time', 'technology']
verbs = ['explore', 'discover', 'communicate']
adjectives = ['strange', 'advanced', 'intelligent']

# Get embeddings from the model
embeddings = model3.embeddings

# Find and print closest words
for category, words in [('Nouns', nouns), ('Verbs', verbs), ('Adjectives', adjectives)]:
    print(f"\n{category}:")
    for word in words:
        closest = get_closest_words(word, word_to_ix2, embeddings)
        print(f"{word}: {closest}")


Nouns:
robot: ['beautician', 'outraged', 'formallylet', 'dean', 'server']
time: ['posession', 'seventyone', 'polyculture', 'adthus', 'problemstory']
technology: ['recapturing', 'beachcomber', 'longerrange', 'stalkinghorse', 'refound']

Verbs:
explore: ['stuns', 'paiagraphs', 'institutionalized', 'missle', 'monstercon']
discover: ['unemployed', 'favour', 'restatement', 'hyperdimensional', 'suggest']
communicate: ['prearranging', 'foostmams', 'nonpol', 'benign', 'expuritan']

Adjectives:
strange: ['rallying', 'sunlike', 'bong', 'vsrill', 'sideaffects']
advanced: ['theretofore', 'yital', 'transcription', 'jsland', 'yourc']
intelligent: ['extrapsi', 'physical', 'inferior', 'emotional', 'coherency']


### 2.3 🗒❓ How does the quality of the hotel review-based embeddings compare with the Sci-fi-based embeddings? Elaborate.

### 2.4 Choose 2 words and retrieve their 5 closest neighbours according to hotel review-based embeddings and the Sci-fi-based embeddings.

🗒❓ Do they have different neighbours? If yes, can you reason why?

In [21]:



words_to_compare = ['room', 'star']

# Hotel review-based embeddings
for word in words_to_compare:
    closest_words = get_closest_words(word, word_to_ix, model1.embeddings) #LATER CHANGE TO ***model1***
    print(f"Closest words to '{word}' in Hotel Reviews: {closest_words}")

# Sci-fi-based embeddings
for word in words_to_compare:
    closest = get_closest_words1(word, word_to_ix2, model3.embeddings) # LATER CHANGE TO ***model3*** # Use word_to_ix2 for Sci-fi
    print(f"Closest words to '{word}' in Sci-Fi: {closest}")


Closest words to 'room' in Hotel Reviews: ['31c', 'unit', 'intranet', 'expo', 'everybodythe']
Closest words to 'star' in Hotel Reviews: ['nomad', 'preminim', 'grecoroman', 'groom', 'bad']
Closest words to 'room' in Sci-Fi: ['junnabls', 'taunton', 'foothigh', 'side', 'cassiopeia']
Closest words to 'star' in Sci-Fi: ['quantification', 'enodimous', 'cashable', 'lactame', 'libration']


### 2.5 🗒❓ What are the differences between CBOW2 and CBOW5 ? Can you "describe" them?    

### Report
The lab report should contain a detailed description of the approaches you have used to solve this exercise. Please also include results.

Answers for the questions marked 🗒❓ goes here as well